In [1]:
import glob

filepaths_to_be_wrapped = []
for i in range(1, 2):
    for file_name in set(glob.glob(str(i) + 'run/*.pdb')) - set(glob.glob(str(i) + 'run/*wrapped*')):
        filepaths_to_be_wrapped.append(file_name)
print(filepaths_to_be_wrapped)


['1run\\snapshot_2.pdb', '1run\\118000_checkpoint.pdb']


In [2]:
for file_name in filepaths_to_be_wrapped:
    temp_newfilename = file_name.split('.')
    newfilename = temp_newfilename[0] + '_wrapped.' + temp_newfilename[1]
    with open(file_name, 'r') as f:
        water_index_list, acrolein_index_list = [], []
        lines = f.readlines()
        for i in range(len(lines)):
            if 'CRYST1' in lines[i]:
                need = lines[i].split()
                x_bound, y_bound, z_boudn = float(need[1]), float(need[2]), float(need[3])
            if 'ATOM' in lines[i]:
                if ' O ' in lines[i]:
                    if ' H ' in lines[i+1]:
                        water_index_list.append(i)
                    if ' C ' in lines[i+1]:
                        acrolein_index_list.append(i)
            f.close()

In [3]:
def subtract_atom(molecule, index, bounds):
    for i in range(len(molecule)):
        molecule[i][index] = round(float(molecule[i][index]) - bounds, 3)
    return molecule

def add_atom(molecule, index, bounds):
    for i in range(len(molecule)):
        molecule[i][index] = round(float(molecule[i][index]) + bounds, 3)
    return molecule

In [4]:
molecules = []
translated_molecules = []
for i in range(len(water_index_list)):
    molecule = []
    out_of_bounds_list = []
    for x in range(3):
        atom = lines[water_index_list[i] + x].split()
        atom = [float(atom[u]) for u in range(5, 8)]
        molecule.append(atom)
    molecules.append(molecule)
    molecule_in_bounds = 1
    for x in range(len(molecules)):
        for y in range(len(molecules[x][0])):
            coordinate = molecules[x][0][y]
            if coordinate < 0:
                #print('old_molecule')
                #print(molecules[x])
                new_molecule = add_atom(molecule = molecules[x], index = y, bounds = x_bound)
                #print('new_molecule')
                #print(new_molecule)
            if coordinate > x_bound:
                #print('old_molecule')
                print(molecules[x])
                new_molecule = subtract_atom(molecule = molecules[x], index = y, bounds = x_bound)
                #print('new_molecule')
                #print(new_molecule)
                
            if coordinate < x_bound and coordinate > 0:
                translated_molecules.append(new_molecule)
print(molecules)
print(translated_molecules)
print(len(molecules))
print(len(translated_molecules))
print('ayy')


NameError: name 'new_molecule' is not defined